In [9]:
# Import required modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import plotly.express as px

# Ticker list to download
tickers = 'SPY QQQ TLT AAPL MSFT GOOG AMZN NFLX NVDA TSLA'


# Have some fun here and explore what stocks this strategy works for and why...
ticker = 'SPY'
# ticker = 'TLT'
# ticker = 'NVDA'
# ticker = 'AAPL'
# ticker = 'INTC'
# Download data from Yahoo-Finance

data = yf.download(
    tickers = ticker,
    start="2020-01-01", # From what point do we want to start getting stock data
    end="2024-04-30", # From what point do we want to stop getting stock data
    interval = "1d", # The sample rate of the data one stock data every day
    ignore_tz=True,
    auto_adjust=True, # Adjust all fields by splits and dividends
)

results = []
for fast_ma in range(5,50,5):
    for slow_ma in range(30, 300, 5):
        if fast_ma >= slow_ma:
            continue
        else:
            close_stock = data['Close'].copy()
            close_stock = pd.DataFrame(close_stock)
            close_stock.columns = ['close']
            close_stock['daily_change'] = close_stock.close.pct_change().fillna(0)
            close_stock['slow_ma'] = close_stock.close.rolling(window = slow_ma).mean()
            close_stock['fast_ma'] = close_stock.close.rolling(window = fast_ma).mean()
            close_stock = close_stock[close_stock.notnull().all(1)]

            close_stock = close_stock.assign(
                signal = lambda x: np.where(x.fast_ma > x.slow_ma, 1, -1)
            )
            close_stock['signal'] = close_stock['signal'].shift(1,fill_value = 0)
            close_stock['MA_strategy'] = close_stock['daily_change'] * close_stock['signal']

            cumprod_buy_and_hold = (1 + close_stock['daily_change']).cumprod()
            cumprod_strategy = (1 + close_stock['MA_strategy']).cumprod()
            max_peaks = cumprod_strategy.cummax()
            drawdown = (cumprod_strategy - max_peaks) / max_peaks

            annualized_returns = (1 + (close_stock[['daily_change', 'MA_strategy']]).mean())**252 - 1
            annualized_std_deviation = close_stock[['daily_change', 'MA_strategy']].std() * np.sqrt(252)
            max_drawdown = (drawdown.min() - 1) * -1

            perf = dict(
                fast_ma = fast_ma,
                slow_ma = slow_ma,
                annualized_returns = annualized_returns['MA_strategy'],
                annualized_std_deviation = annualized_std_deviation['MA_strategy'],
                annualized_returns_buy_and_hold = annualized_returns['daily_change'],
                max_drawdown = max_drawdown,
                cumprod_buy_and_hold = cumprod_buy_and_hold.iloc[-1],
                cumprod_strategy = cumprod_strategy.iloc[-1],
                strategy_sharpe_ratio = (annualized_returns['MA_strategy'] - 0.05) / annualized_std_deviation['MA_strategy'],
                buy_hold_sharpe_ratio = (annualized_returns['daily_change'] - 0.05) / annualized_std_deviation['daily_change']
            )

            results.append(perf)
            
results = pd.DataFrame(results)
results.sort_values('annualized_returns', ascending=False).head()
results.sort_values('strategy_sharpe_ratio', ascending=False).head()

eval_metric = 'strategy_sharpe_ratio'

df_mat = results.pivot(index = 'fast_ma', columns = 'slow_ma', values=eval_metric)

fig = px.imshow(df_mat,
                color_continuous_scale = 'RdYlGn',
                aspect = 'auto')

fig.update_layout(
    title=eval_metric,
    xaxis_title='Slow MA',
    yaxis_title='Fast MA',
)

fig

# Get our optimised moving average windows
idx = results['strategy_sharpe_ratio'].idxmax()
fast_ma = results.loc[idx, 'fast_ma']
slow_ma = results.loc[idx, 'slow_ma']

# get close adjusted prices
data.head()
data = yf.download(
  tickers = ticker,
  start="2020-01-01", # From what point do we want to start getting stock data
  end="2024-04-30", # From what point do we want to stop getting stock data
  interval = "1d", # The sample rate of the data one stock data every day
  ignore_tz=True,
  auto_adjust=True, # Adjust all fields by splits and dividends
)
close_stock = data['Close'].copy()
close_stock = pd.DataFrame(close_stock)
close_stock.columns = ['close']
close_stock['daily_change'] = close_stock.close.pct_change().fillna(0)
close_stock['slow_ma'] = close_stock.close.rolling(window = slow_ma).mean()
close_stock['fast_ma'] = close_stock.close.rolling(window = fast_ma).mean()
close_stock = close_stock[close_stock.notnull().all(1)]
close_stock = close_stock.assign(
                signal = lambda x: np.where(x.fast_ma > x.slow_ma, 1, -1)
            )
close_stock['signal'] = close_stock['signal'].shift(1,fill_value = 0)
close_stock['MA_strategy'] = close_stock['daily_change'] * close_stock['signal']

# Get the cumulative product over the 4 years for both our strategy and when we just buy and hold
cumprod_strategy = (1 + close_stock['MA_strategy']).cumprod()
cumprod_buy_and_hold = (1 + close_stock['daily_change']).cumprod()
cumprod_signals = pd.DataFrame(dict(
    strategy = cumprod_strategy,
    buy_hold = cumprod_buy_and_hold
))

# Scale our money by the cumulative products
px.line(100 * cumprod_signals)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
